### Currency Conversion Tool

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool, InjectedToolArg
from langchain_core.messages import HumanMessage
from typing import Annotated
import requests
from dotenv import load_dotenv

In [2]:
# tool creation - currency conversion 

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """This tool returns the conversion factor from one currency to another."""
    url = f"https://v6.exchangerate-api.com/v6/1a8d353e882a0e61bf06d928/pair/{base_currency}/{target_currency}"
    response = requests.get(url)
    return response.json()
    

In [3]:
# tool for currency calculation:
# Annotated[float, InjectedToolArg] = Developer will specify the argument for conversion_rate during runtime

@tool
def calculate_currency_amount(amount: float, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """This tool returns the converted currency amount based on the conversion rate and the input amount."""
    converted_amount = amount * conversion_rate
    return converted_amount

In [4]:
get_conversion_factor.invoke({"base_currency":"USD", "target_currency":"EUR"})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1761868802,
 'time_last_update_utc': 'Fri, 31 Oct 2025 00:00:02 +0000',
 'time_next_update_unix': 1761955202,
 'time_next_update_utc': 'Sat, 01 Nov 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'EUR',
 'conversion_rate': 0.8635}

In [5]:
calculate_currency_amount.invoke({"amount": 100, "conversion_rate": 0.8604})

86.04

In [6]:
# tool binding steps

tools = [
    get_conversion_factor,
    calculate_currency_amount
]

load_dotenv()

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

llm_with_tools = llm.bind_tools(tools)


In [7]:
# tool calling steps

messages = [
    HumanMessage(content="What is the conversion rate from USD to EUR? And how much is 100 USD in EUR?")
]

output = llm_with_tools.invoke(messages)

messages.append(output)
print(output.tool_calls)   

[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'EUR'}, 'id': 'call_xJZ0vZGhBi9c2PmSbDFqFABm', 'type': 'tool_call'}, {'name': 'calculate_currency_amount', 'args': {'amount': 100}, 'id': 'call_bxE0Nam8yCdpG8iwGGIENEGI', 'type': 'tool_call'}]


In [8]:
import json

for tool_call in output.tool_calls:
    # execute the first tool and get the conversion rate
    if tool_call["name"] == "get_conversion_factor":
        tool_message_1 = get_conversion_factor.invoke(tool_call)
        print(tool_message_1)
        # fetch conversion rate from the response
        conversion_rate = json.loads(tool_message_1.content)["conversion_rate"]

        # append the tool_message_1 to messages
        messages.append(tool_message_1)

    if tool_call["name"] == "calculate_currency_amount":
        # first add the conversion rate to the tool_call
        tool_call["args"]["conversion_rate"] = conversion_rate
        # execute the second tool and get the converted amount
        tool_message_2 = calculate_currency_amount.invoke(tool_call)
        print(tool_message_2)

        # append the tool_message_2 to messages
        messages.append(tool_message_2)

print(messages)

content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1761868802, "time_last_update_utc": "Fri, 31 Oct 2025 00:00:02 +0000", "time_next_update_unix": 1761955202, "time_next_update_utc": "Sat, 01 Nov 2025 00:00:02 +0000", "base_code": "USD", "target_code": "EUR", "conversion_rate": 0.8635}' name='get_conversion_factor' tool_call_id='call_xJZ0vZGhBi9c2PmSbDFqFABm'
content='86.35000000000001' name='calculate_currency_amount' tool_call_id='call_bxE0Nam8yCdpG8iwGGIENEGI'
[HumanMessage(content='What is the conversion rate from USD to EUR? And how much is 100 USD in EUR?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xJZ0vZGhBi9c2PmSbDFqFABm', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "EUR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_bxE0Nam8yCdpG8iwGGIE

In [9]:
llm_with_tools.invoke(messages).content

'The conversion rate from USD to EUR is 0.8635. \n\nTherefore, 100 USD is equivalent to 86.35 EUR.'